In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from ast import literal_eval
from sklearn.preprocessing import Imputer
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import warnings; warnings.simplefilter('ignore')

# Problema
+ O problema proposto é um mecanismo para recomendação de filmes. Esse mecanismo deve ser capaz de dado uma entrada pelo usuário(nome de um filme) o sistema seja capaz de recomendar 5 novos filmes semelhantes ao que o usuário informou.  
  
# Funcionamento
+ A recomendação é baseada pelo conteudo do filme(diretor, titulo, ano de lançamento, atores, palavras, entre outros), dessa forma o usuário informa um filme e apartir dessa entrada será calculada a semelhança desse filme com o restante do dataset, a priori será formado uma lista com 50 filmes, dessa lista vai ser recomendado ao usuário 5 filmes mais semelhantes ao que ele informou.  

# Dados
+ Os dados disponibilizados contêm metadados para todos os 45.000 filmes listados no Conjunto de Dados Completo do MovieLens. O conjunto de dados consiste em filmes lançados em julho de 2017 ou antes. Os dados incluem elenco, equipe, palavras-chave, orçamento, receita, cartazes, datas de lançamento, idiomas, empresas de produção, países, contagem de votos TMDB e médias de votos.  
+ Este conjunto de dados também possui arquivos contendo 26 milhões de avaliações de 270.000 usuários para todos os 45.000 filmes. As classificações estão em uma escala de 1 a 5 e foram obtidas no site oficial do GroupLens.  

## Datasets Originais  
**movies_metadata.csv:** O principal arquivo de Metadados de Filmes. Contém informações sobre 45.000 filmes apresentados no conjunto de dados Full MovieLens. Os recursos incluem cartazes, panos de fundo, orçamento, receita, datas de lançamento, idiomas, países de produção e empresas.

**keywords.csv:** Contém as palavras-chave do enredo do filme para nossos filmes do MovieLens. Disponível na forma de um Objeto JSON com string.

**credits.csv:** Consiste em informações sobre o elenco e a equipe para todos os nossos filmes. Disponível na forma de um Objeto JSON com string.

**links.csv:** O arquivo que contém os IDs TMDB e IMDB de todos os filmes apresentados no conjunto de dados Full MovieLens.

**links_small.csv:** Contém os IDs TMDB e IMDB de um pequeno subconjunto de 9.000 filmes do Dataset Completo.

**ratings_small.csv:** O subconjunto de 100.000 avaliações de 700 usuários em 9.000 filmes.  

## Datasets Reduzidos  
**tmdb_5000_credits.csv:** Contém dados sobre o elenco e a equipe técnica do filme, possui dados de 5.000 mil filmes.  

**tmdb_5000_movies.csv:** Contém informações sobre os filmes, tais como, titulo, ano de lançamento, media de votos, entre outros.


  
  

# Tipo de Problema
+ Problema de recomendação

# Features Usadas
+ Como é um problema de aprendizado não supervisionado, logo não existe uma label(output). A seguir as features que vão ser usadas: 'index', 'genres', 'id', 'keywords', 'language', 'original_title', 'overview', 'popularity', 'spoken_languages', 'tagline', 'title', 'vote_average', 'num_voted_users', 'title_year', 'country', 'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'description'. Muitas não são do dataset original, foram derivadas.

# Metricas de Avaliação Usadas
+ Classificação Ponderada  
+ Distância Euclidiana  
+ Gaussian  
+ Cosine

## Pré-processamento dos Dados

**load_movies():** Função responsável por fazer o carregamento do arquivo que contem os dados sobre os filmes.

In [44]:
# Função responsável por carregar o arquivo csv com os dados no formato json
def load_movies(path):
    # Aqui é feito o carregamento do arquivo 'movies' csv
    data = pd.read_csv(path)
    # Aqui é aplicado uma função lambda na coluna que possui a data de lançamento
    data['release_date'] = pd.to_datetime(data['release_date']).apply(lambda x: x.date())
    # Aqui é gerado algumas colunas que seu formato no dataset é json
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    # Aqui é carrregado as colunas com formato json com a função json.loads
    for column in json_columns:
        data[column] = data[column].apply(json.loads)
    return data

**load_credits:** Essa função função é responsavel por fazer o carregamento do arquivo de creditos, onde contem informações importantes sobre os filmes, tais como: Atores, diretor e etc.

In [45]:
def load_credits(path):
    # Aqui é feito o carregamento do arquivo 'credits' csv
    data = pd.read_csv(path)
    # Aqui é gerado algumas colunas que seu formato no dataset é json
    json_columns = ['cast', 'crew']
    # Aqui é carrregado as colunas com formato json com a função json.loads
    for column in json_columns:
        data[column] = data[column].apply(json.loads)
    return data

+ Aqui é feito apenas a renomeação de algumas colunas do dataset, apenas para melhor visualização.

In [4]:
# Essas colunas são criadas com o objetivo de deixar os dados mais claros, alguns são renomeadas e outras permanecem iguais
COLUMNS_EQUIVALENCIES = {
    'budget': 'budget',
    'genres': 'genres',
    'revenue': 'gross',
    'title': 'title',
    'runtime': 'duration',
    'original_language': 'language',
    'keywords': 'keywords',
    'vote_count': 'num_voted_users',
}

**safe_access():** Função já implementada pelo python, sua função é apenas verificar se uma chave existe em dicionario.

In [5]:
# Essa função verifica se uma chave está em um dicionario
# Essa função já está implementado e pode ser encontrada facilmente
def safe_access(container, index_values):
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan

**get_director():** Essa função é responsavel por retornar o ou os diretores de um filme

In [46]:
# Essa função recebe a coluna crew que está em formato json, e devolve o nome do diretor com ajuda da função safe_access
def get_director(crew):
    directors = [x['name'] for x in crew if x['job'] == 'Director']
    return safe_access(directors, [0])

**get_words()** Aqui é feito apenas uma formatação nas palavras chaves de cada filme

In [47]:
# Essa função devolve as palavras chaves separadas por pipe(|), antes estavam em formato json 
def get_words(words):
    return '|'.join([x['name'] for x in words])

**convert():** Aqui é feito a união do dataset movies e com o dataset credicts.

In [48]:
# Essa função é responsavel por unir os dois datasets, dessa forma fica melhor na hora em que for realizar a recomendação.
def convert(dataset_movies, dataset_credits):
    # Aqui o dataset movies é copiado para a variavel movies_tmdb
    movies = dataset_movies.copy()
    # Aqui as colunas do dataset que está em movies_tmdb são renomeadas de acordo com as colunas definidas anteriormente.
    # o parametro inplace garante que a alteração seja feita na versão do dataset que está em memoria
    movies.rename(columns=COLUMNS_EQUIVALENCIES, inplace=True)
    # Aqui é adicionado a nova coluna 'title_year' onde está sendo gerada apartir do ano de lançamento do filme
    movies['title_year'] = pd.to_datetime(movies['release_date']).apply(lambda x: x.year)
    
    # Aqui assumi(baseado em uma leitura que fiz) que o primeiro pais que aparece na coluna é o pais de produção
    # É criada a nova conluna country e preenchida com o pais de produção
    movies['country'] = movies['production_countries'].apply(lambda x: safe_access(x, [0, 'name']))
    # Aqui só é pegado a primeira linguagem que aparece na coluna
    movies['language'] = movies['spoken_languages'].apply(lambda x: safe_access(x, [0, 'name']))
    # Aqui é identificado o nome do diretor
    movies['director_name'] = dataset_credits['crew'].apply(get_director)
    movies['actor_1_name'] = dataset_credits['cast'].apply(lambda x: safe_access(x, [1, 'name']))
    movies['actor_2_name'] = dataset_credits['cast'].apply(lambda x: safe_access(x, [2, 'name']))
    movies['actor_3_name'] = dataset_credits['cast'].apply(lambda x: safe_access(x, [3, 'name']))
    movies['genres'] = movies['genres'].apply(get_words)
    movies['keywords'] = movies['keywords'].apply(get_words)
    return movies

+ Aqui é carregado os datasets

In [49]:
# Carregamento dos dados
dataset_credits = load_credits("tmdb-5000-movie-dataset/tmdb_5000_credits.csv")
dataset_movies = load_movies("tmdb-5000-movie-dataset/tmdb_5000_movies.csv")
dados = convert(dataset_movies, dataset_credits)

Na proxima celula é feita a remoção de colunas do dataset que são de baixa relevancia para a resolução do problema. Essas colunas são removidas, pois podem prejudicar o resultado final.

In [50]:
# Aqui é feito a remoção de colunas julgadas de pouca relevância
dados = dados.drop(['budget', 'homepage', 'gross', 'status', 'duration', 'production_countries', 
                    'release_date', 'production_companies'], axis=1);

In [51]:
dados['title_year'] = dados['title_year'].values.astype(np.int64)

#### O dataset final

In [52]:
dados.head(3)

,genres,id,keywords,language,original_title,overview,popularity,spoken_languages,tagline,title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,Action|Adventure|Fantasy|Science Fiction,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Enter the World of Pandora.,Avatar,7.2,11800,2009,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,Adventure|Fantasy|Action,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'iso_639_1': 'en', 'name': 'English'}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,Action|Adventure|Crime,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",A Plan No One Escapes,Spectre,6.3,4466,2015,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes


## Top 10 dos filmes mais bem avaliados.
Aqui é utilizada a formula de classificação ponderada do IMDB's, para construir essa classificação. A formula é representada assim:  


\begin{equation}
     \textbf{Classificação Ponderada} = (\frac{v}{v + m} * R) + (\frac{m}{v + m} * C)
\end{equation}  

* V é numero de votos para o filme
* M é o mínimo de votos necessários para o filme ser listado
* R é a classificação média do filme
* C é a média de votos em todo o dataset


A seguir é calculado o parametro C, no qual é a media dos votos de todos os filmes. E tambem é calculado a quantidade de votos que cada filme obteve.

In [14]:
vote_counts = dados[dados['num_voted_users'].notnull()]['num_voted_users'].astype('int')
vote_averages = dados[dados['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
print("Media de votos de todos os filmes: ", C, '\n')

Media de votos de todos os filmes:  5.6529252550489275 



**Nesse proximmo passo é definido um valor minimo de votos para o filme ser classificado (M). Foi definido como parametro de seleção um percentil de 95%, ou seja, para um filme ser classificado ele deve ter mais votos do que 95% da lista toda.**

In [53]:
minimo = vote_counts.quantile(0.95)
print("Valor minimo de votos: ", minimo, '\n')

Valor minimo de votos:  3040.8999999999996 



**Portanto, para se qualificar para ser mostrado no top10, um filme precisa ter pelo menos 3040 votos no TMDB. Também vemos que a classificação média de um filme no TMDB é de 5.652 em uma escala de 10. Os 241 filmes se qualificam para estar em nosso top10 (A seguir é calculado a quantidade de filmes que se qualificam).**

In [54]:
qualified = dados[(dados['num_voted_users'] >= minimo) & (dados['num_voted_users'].notnull()) & (dados['vote_average'].notnull())][['title', 'title_year', 'num_voted_users', 'vote_average', 'popularity', 'genres']]
qualified['num_voted_users'] = qualified['num_voted_users'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(241, 6)

**A seguir é definida a formula para o calculo da classificação.**

In [55]:
def weighted_rating(x):
    v = x['num_voted_users']
    R = x['vote_average']
    return (v / (v + minimo) * R) + (minimo / (minimo + v) * C)

**Agora vamos criar uma nova coluna onde vai estar o valor da clssificação ponderada para os 241 filmes qualificados.**

In [56]:
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('weighted_rating', ascending=False).head(241)

### Agora podemos ver nosso top10

In [57]:
qualified.head(10)

,title,title_year,num_voted_users,vote_average,popularity,genres,weighted_rating
96,Inception,2010,13752,8,167.583710,Action|Thriller|Science Fiction|Mystery|Adventure,7.574986
65,The Dark Knight,2008,12002,8,187.322927,Drama|Action|Crime|Thriller,7.525542
95,Interstellar,2014,10867,8,724.247784,Adventure|Drama|Science Fiction,7.486823
662,Fight Club,1999,9413,8,146.757391,Drama,7.426909
262,The Lord of the Rings: The Fellowship of the Ring,2001,8705,8,138.049577,Adventure|Fantasy|Action,7.392365
3232,Pulp Fiction,1994,8428,8,121.463076,Thriller|Crime,7.377689
1881,The Shawshank Redemption,1994,8205,8,136.747729,Drama|Crime,7.365349
329,The Lord of the Rings: The Return of the King,2003,8064,8,123.630332,Adventure|Fantasy|Action,7.357291
809,Forrest Gump,1994,7927,8,138.133331,Comedy|Drama|Romance,7.349263
330,The Lord of the Rings: The Two Towers,2002,7487,8,106.914973,Adventure|Fantasy|Action,7.322066


# Sistema de Recomendação

### Funções básicas:
    1. Determinar N filmes com um conteúdo semelhante ao fornecido pelo usuário.
    2. Selecionar os 5 filmes mais populares entre esses N filmes.

**Calculando a Similaridade**  
    A primeira coisa a fazer é definir um criterio que compute o quão dois filmes são próximos. Para isso, é necessário extrair algumas informações do filme selecionado pelo usuário, tais como: nome do diretor, os nomes dos atores e algumas palavras-chaves. Com isso construo uma matriz onde cada linha corresponde a um filme do dataset e onde as colunas correspondem às informações anteriores (diretor + atores + palavras-chave) mais os k gêneros.  
    
Nesta matriz, os coeficientes Mij tomam o valor 0 ou 1 dependendo da correspondência entre o significado da coluna j e o conteúdo do filme i. Por exemplo, se "palavra-chave1" estiver no filme i, teremos Mij = 1 e 0 caso contrário. Uma vez que esta matriz tenha sido definida, determinamos a distância entre dois filmes de acordo com:  

\begin{equation}
     d_{j,n} = \sqrt{\sum_{i=1}^{N} (M_{j,i} - M_{n,i})^2}
\end{equation}  



**Calculando a Popularidade**  
De acordo com as semelhanças entre o filme dado como entrada e o restante do dataset é gerado uma lista de N=50 filmes. Dessa lista seleciono 5, baseado em uma nota que será calculada com os seguintes criterios:
+ A pontuação do IMDB
+ O número de votos que o filme recebeu
+ O ano do lançamento  

A nota vai ser calculada com a seguinte formula:
\begin{equation}
     score = IMDB^2\ \times\ \phi_{\sigma_{1}, c_{1}}\ \times\ \phi_{\sigma_{2}, c_{2}}
\end{equation}  

Onde $\phi$ é uma função Gaussian:  
\begin{equation}
     \phi_{\sigma, c}(x)\ \infty\ exp \bigg(-\frac{(x - c)^2}{2\sigma^2}\bigg)
\end{equation}

### Definição da Função Gaussian

In [20]:
gaussian = lambda x, y, sigma: np.exp(-(x - y) ** 2 / (2 * sigma ** 2))

**A função capture_director_actor_keywords()** retorna os valores tomados pelas colunas 'director_name', 'actor_N_name' e 'keywords' para o filme selecionado pelo usuário.

In [21]:
def capture_director_actor_keywords(df, id_): 
    col_labels = []    
    if pd.notnull(df['director_name'].iloc[id_]):
        for name in df['director_name'].iloc[id_].split('|'):
            col_labels.append(name)
            
    for i in range(3):
        column = 'actor_NUM_name'.replace('NUM', str(i + 1))
        if pd.notnull(df[column].iloc[id_]):
            for name in df[column].iloc[id_].split('|'):
                col_labels.append(name)
                
    if pd.notnull(df['keywords'].iloc[id_]):
        for word in df['keywords'].iloc[id_].split('|'):
            col_labels.append(word)
    return col_labels

**A função add_variables()** adiciona uma lista de variáveis ao dataset dado na entrada e inicializa estas variáveis com 0 ou 1 dependendo da correspondência com a descrição dos filmes e o conteúdo da variável var dada na entrada.

In [22]:
def add_variables(df, var):    
    for s in var: df[s] = pd.Series([0 for _ in range(len(df))])
    colonnes = ['genres', 'actor_1_name', 'actor_2_name',
                'actor_3_name', 'director_name', 'keywords']
    for categorie in colonnes:
        for index, row in df.iterrows():
            if pd.isnull(row[categorie]): continue
            for name in row[categorie].split('|'):
                if name in var: df.set_value(index, name, 1)            
    return df

 **A função recommand()** cria uma lista de N=50 filmes similares ao filme selecionado pelo usuário.

In [23]:
def recommand(df, id_):    
    df_copy = df.copy(deep = True)    
    liste_genres = set()
    for s in df['genres'].str.split('|').values:
        liste_genres = liste_genres.union(set(s))    
    # Criada uma variavel adicional para chegar a similaridade
    variables = capture_director_actor_keywords(df_copy, id_)
    variables += list(liste_genres)
    df_new = add_variables(df_copy, variables)

    X = df_new.as_matrix(variables)
    model_nbrs = NearestNeighbors(n_neighbors=50, algorithm='auto', metric='euclidean').fit(X)

    distances, indices = model_nbrs.kneighbors(X)    
    xtest = df_new.iloc[id_].as_matrix(variables)
    xtest = xtest.reshape(1, -1)

    distances, indices = model_nbrs.kneighbors(xtest)
    return indices[0][:]

**A função sequel()** compara 2 títulos de filmes dados na entrada e define se estes títulos são semelhantes ou não.

In [24]:
def sequel(titre_1, titre_2):    
    if fuzz.ratio(titre_1, titre_2) > 50 or fuzz.token_set_ratio(titre_1, titre_2) > 50:
        return True
    else:
        return False

**A função critere_selection()** da uma nota a um filme, dependendo da sua pontuação no IMDB, o ano do título e o número de usuários que votaram para este filme.

In [25]:
def critere_selection(title_main, max_users, year_ref, titre, year, imdb_score, votes):    
    if pd.notnull(year_ref):
        facteur_1 = gaussian(year_ref, year, 20)
    else:
        facteur_1 = 1        

    sigma = max_users * 1.0

    if pd.notnull(votes):
        facteur_2 = gaussian(votes, max_users, sigma)
    else:
        facteur_2 = 0
        
    if sequel(title_main, titre):
        note = 0
    else:
        note = imdb_score ** 2 * facteur_1 * facteur_2
    return note

**A função extract_parameters()** extrai algumas variáveis do dataset dado na entrada e retorna uma lista para a seleção de N filmes. Esta lista é ordenada de acordo com os critérios estabelecidos na função critere_selection().

In [26]:
def extract_parameters(df, list_films):     
    parametres_films = ['_' for _ in range(50)]
    i = 0
    max_users = -1
    for index in list_films:
        parametres_films[i] = list(df.iloc[index][['title', 'title_year', 'imdb_score', 'num_user_for_reviews', 'num_voted_users']])
        parametres_films[i].append(index)
        max_users = max(max_users, parametres_films[i][4] )
        i += 1
        
    title_main = parametres_films[0][0]
    year_ref  = parametres_films[0][1]
    parametres_films.sort(key = lambda x: critere_selection(title_main, max_users, year_ref, x[0], x[1], x[2], x[4]), reverse = True)
    return parametres_films 

**A função add_to_selection()** completa a lista film_selection que contém 5 filmes que serão recomendados ao usuário. Os filmes são selecionados na lista parametres_films.

In [27]:
def add_to_selection(film_selection, parametres_films):    
    film_list = film_selection[:]
    count = len(film_list)    
    for i in range(50):
        already_in_list = False
        for film in film_selection:
            if film[0] == parametres_films[i][0]: already_in_list = True
            if sequel(parametres_films[i][0], film[0]): already_in_list = True            
        if already_in_list: continue
        count += 1
        if count <= 5:
            film_list.append(parametres_films[i])
    return film_list

**A função remove_sequels()** remove os filmes da lista se mais de dois filmes de uma mesma série estiverem presentes. O mais antigo é mantido.

In [28]:
def remove_sequels(film_selection):    
    removed_from_selection = []
    for i, film_1 in enumerate(film_selection):
        for j, film_2 in enumerate(film_selection):
            if j <= i: continue 
            if sequel(film_1[0], film_2[0]): 
                last_film = film_2[0] if film_1[1] < film_2[1] else film_1[0]
                removed_from_selection.append(last_film)

    film_list = [film for film in film_selection if film[0] not in removed_from_selection]
    return film_list

**Função find_similarities()** cria uma lista com 5 filmes e recomenda ao usuário

In [29]:
def find_similarities(df, id_, del_sequels=True, verbose=False):    
    if verbose: 
        print('\n' + "CONSULTA: Filmes similares ao id={} -> '{}'".format(id_, df.iloc[id_]['title']))

    liste_films = recommand(df, id_)
    # Criada a lista com 50 filmes
    parametres_films = extract_parameters(df, liste_films)
    # Selecionado 5 filmes da lista com 50
    film_selection = []
    film_selection = add_to_selection(film_selection, parametres_films)
    # Excluindo repetidos, se houver
    if del_sequels: film_selection = remove_sequels(film_selection)
    # Adicionando novos filmes a lista, caso tenha sido excluido algum
    film_selection = add_to_selection(film_selection, parametres_films)

    selection_titres = []
    for i, x in enumerate(film_selection):
        selection_titres.append([x[0].replace(u'\xa0', u''), x[5]])
        if verbose: print("nº{:<2}     -> {:<50}".format(i + 1, x[0]))

    return selection_titres

In [30]:
aux = find_similarities(dados, 0, del_sequels=False, verbose=True)


CONSULTA: Filmes similares ao id=0 -> 'Avatar'
nº1      -> Avatar                                            
nº2      -> Star Trek Into Darkness                           
nº3      -> John Carter                                       
nº4      -> Pirates of the Caribbean: On Stranger Tides       
nº5      -> The Fifth Element                                 


In [31]:
aux = find_similarities(dados, 0, del_sequels=True, verbose=True)


CONSULTA: Filmes similares ao id=0 -> 'Avatar'
nº1      -> Avatar                                            
nº2      -> Star Trek Into Darkness                           
nº3      -> John Carter                                       
nº4      -> Pirates of the Caribbean: On Stranger Tides       
nº5      -> The Fifth Element                                 


In [32]:
dados.columns

Index(['genres', 'id', 'keywords', 'language', 'original_title', 'overview',
       'popularity', 'spoken_languages', 'tagline', 'title', 'vote_average',
       'num_voted_users', 'title_year', 'country', 'director_name',
       'actor_1_name', 'actor_2_name', 'actor_3_name'],
      dtype='object')

In [33]:
temp = dados['id']

In [34]:
temp['tagline'] = dados['tagline'].fillna('')
temp['description'] = dados['overview'] + dados['tagline']

dados['description'] = temp['description'].fillna('')

In [35]:
model_tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = model_tf.fit_transform(dados['description'])

**Cosine de Similaridade**  
Usando a semelhança Cosine para calcular a semelhança entre dois filmes. Definida da seguinte forma:  
\begin{equation}
     cosine(x, y) = \frac{x \cdot y^T}{\left \| x \right \| \cdot \left \| y \right \|}
\end{equation}  

Uma vez que usamos o Vectorizador TF-IDF, ele fornecerá diretamente o Índice de Similaridade do Cosine.

In [36]:
cosine = linear_kernel(tfidf_matrix, tfidf_matrix)

In [37]:
dados = dados.reset_index()
titles = dados['title']
indices = pd.Series(dados.index, index=dados['title'])

In [38]:
def get_recommendations(title):
    idx = indices[title]
    scores = list(enumerate(cosine[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    scores = scores[1:31]
    movie_indices = [i[0] for i in scores]
    return titles.iloc[movie_indices]

In [40]:
get_recommendations('Avatar').head(5)

634                                     The Matrix
3604                                     Apollo 18
529                               Tears of the Sun
369     Lara Croft Tomb Raider: The Cradle of Life
312                                     Green Zone
Name: title, dtype: object